In [ ]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai langchain-groq python-dotenv langchain beautifulsoup4

In [ ]:
import getpass
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

## Define Models

In [ ]:
# from langchain_openai import ChatOpenAI # type: ignore
# model = ChatOpenAI(model="gpt-4o-mini")

from langchain_groq import ChatGroq # type: ignore

model = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


model_with_tools = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Agent Workflow

In [ ]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
)
from langchain_core.messages import BaseMessage 
from langgraph.graph.message import add_messages 


class AgentState(TypedDict):
    """The state of the agent."""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    review_type: str
    db_type: str